[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/drbob-richardson/stat220/blob/main/Assignments/Stat_220_HW9.ipynb)

**Problem 1** A number of measurements were collected for a large variety of cars in order to predict fuel efficiency of the car and to understand what factors significantly affect fuel efficiency. You can read in this data here
cars = pd.read_csv("https://richardson.byu.edu/220/cars.csv")
More information on this data can be found here: https://code.datasciencedojo.com/datasciencedojo/datasets/tree/master/Auto%20MPG
Your task is to build a model that we can use to determine important information about the relationship between fuel efficiency and the other factors.

Part a. Build and tune a linear regression model to predict the target variable, mpg or miles per gallon, using all the other variables besides car_name as predictors.  Tune the model so that only significant features remain.

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

cars = pd.read_csv("https://richardson.byu.edu/220/cars.csv")

X = cars.select_dtypes(include=[np.number]).drop('mpg', axis=1)
y = cars['mpg']

X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

significant_vars = model.pvalues[model.pvalues < 0.05].index.tolist()
if 'const' in significant_vars:
    significant_vars.remove('const')

X_sig = X[significant_vars]
X_sig_const = sm.add_constant(X_sig)
model_tuned = sm.OLS(y, X_sig_const).fit()

Part b. You will likely find the variable weight significant. What is the confidence interval for the coefficient associated with weight?

In [3]:

ci = model_tuned.conf_int()
weight_ci = ci.loc['weight']

print(f"Confidence interval for weight: [{weight_ci[0]:.4f}, {weight_ci[1]:.4f}]")

Confidence interval for weight: [-0.0071, -0.0062]


Part c. You are asked, how does weight affect fuel efficiency. Answer the question while providing some uncertainty to the answer (i.e. interpret the confidence interval).

Weight has a negative effect on fuel efficiency. For every 1 pound increase in weight, we can expect mpg to decrease by between 0.0062 and 0.0071 miles per gallon. The confidence interval tells us that we can be about 95% sure that the true mpg decrease is inbetween 0.0062 and 0.0071 miles per gallon.

**Problem 2** Explore adding higher order terms into the model

Part a. Fit and tune a model that includes the interaction between weight and model_year.

In [4]:

X_int = X.copy()
X_int['weight_x_model_year'] = X['weight'] * X['model_year']

X_int_const = sm.add_constant(X_int)
model_int = sm.OLS(y, X_int_const).fit()

sig_vars = model_int.pvalues[model_int.pvalues < 0.05].index.tolist()
if 'const' in sig_vars:
    sig_vars.remove('const')

X_int_sig = X_int[sig_vars]
X_int_sig_const = sm.add_constant(X_int_sig)
model_int_tuned = sm.OLS(y, X_int_sig_const).fit()

Part b. Interpret this model, including the interactions, to a non-technical audience using no numbers.

The model shows that heavier cars generally have worse fuel efficiency, but this effect also depends on the car's model year. For newer cars, the impact of weight on fuel efficiency is less severe than for older cars. Basically, becuase cars are now more modern the technology gets better at handling extra weight, so weight matters less for fuel efficiency in newer cars compared to older ones.

Part c. Examine using all higher order terms in the model. Using out of sample metrics, compare a model with no higher terms to a model with all higher order terms (no reducing the full model yet). Remember to keep car name out of the predictor set.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)
model_no_poly = sm.OLS(y_train, X_train_const).fit()
y_pred_no_poly = model_no_poly.predict(X_test_const)
rmse_no_poly = np.sqrt(mean_squared_error(y_test, y_pred_no_poly))
r2_no_poly = r2_score(y_test, y_pred_no_poly)

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_train_poly_const = sm.add_constant(X_train_poly)
X_test_poly_const = sm.add_constant(X_test_poly)
model_all_poly = sm.OLS(y_train, X_train_poly_const).fit()
y_pred_all_poly = model_all_poly.predict(X_test_poly_const)
rmse_all_poly = np.sqrt(mean_squared_error(y_test, y_pred_all_poly))
r2_all_poly = r2_score(y_test, y_pred_all_poly)

print(f"No higher order terms: RMSE = {rmse_no_poly:.3f}, R² = {r2_no_poly:.3f}")
print(f"All higher order terms: RMSE = {rmse_all_poly:.3f}, R² = {r2_all_poly:.3f}")

No higher order terms: RMSE = 3.241, R² = 0.794
All higher order terms: RMSE = 2.724, R² = 0.855


Part d. Reduce the full model to a set of significant predictors. Compare this reduced model to the previous two models (no higher order terms and all higher order terms).

In [6]:

sig_poly_vars = [i for i, p in enumerate(model_all_poly.pvalues[1:]) if p < 0.05]
X_train_poly_red = X_train_poly[:, sig_poly_vars]
X_test_poly_red = X_test_poly[:, sig_poly_vars]
X_train_poly_red_const = sm.add_constant(X_train_poly_red)
X_test_poly_red_const = sm.add_constant(X_test_poly_red)
model_poly_red = sm.OLS(y_train, X_train_poly_red_const).fit()
y_pred_poly_red = model_poly_red.predict(X_test_poly_red_const)
rmse_poly_red = np.sqrt(mean_squared_error(y_test, y_pred_poly_red))
r2_poly_red = r2_score(y_test, y_pred_poly_red)

print(f"No higher order: RMSE = {rmse_no_poly:.3f}, R² = {r2_no_poly:.3f}")
print(f"All higher order: RMSE = {rmse_all_poly:.3f}, R² = {r2_all_poly:.3f}")
print(f"Reduced (significant only): RMSE = {rmse_poly_red:.3f}, R² = {r2_poly_red:.3f}")

No higher order: RMSE = 3.241, R² = 0.794
All higher order: RMSE = 2.724, R² = 0.855
Reduced (significant only): RMSE = 3.263, R² = 0.791


Part e. Explore building a model using the original variables and adding only a few important higher terms you found from the full model. Again compare this model to the others.

In [10]:

X_train_select = X_train.copy()
X_test_select = X_test.copy()

# add weight squared
X_train_select['weight_squared'] = X_train['weight'] ** 2
X_test_select['weight_squared'] = X_test['weight'] ** 2

# add displacement squared
X_train_select['displacement_squared'] = X_train['displacement'] ** 2
X_test_select['displacement_squared'] = X_test['displacement'] ** 2

# add weight x model_year interaction
X_train_select['weight_x_year'] = X_train['weight'] * X_train['model_year']
X_test_select['weight_x_year'] = X_test['weight'] * X_test['model_year']

X_train_select_const = sm.add_constant(X_train_select)
X_test_select_const = sm.add_constant(X_test_select)
model_select = sm.OLS(y_train, X_train_select_const).fit()
y_pred_select = model_select.predict(X_test_select_const)
rmse_select = np.sqrt(mean_squared_error(y_test, y_pred_select))
r2_select = r2_score(y_test, y_pred_select)

print(f"No higher order: RMSE = {rmse_no_poly:.3f}, R² = {r2_no_poly:.3f}")
print(f"All higher order: RMSE = {rmse_all_poly:.3f}, R² = {r2_all_poly:.3f}")
print(f"Reduced (significant): RMSE = {rmse_poly_red:.3f}, R² = {r2_poly_red:.3f}")
print(f"Select terms (a few key): RMSE = {rmse_select:.3f}, R² = {r2_select:.3f}")

No higher order: RMSE = 3.241, R² = 0.794
All higher order: RMSE = 2.724, R² = 0.855
Reduced (significant): RMSE = 3.263, R² = 0.791
Select terms (a few key): RMSE = 2.722, R² = 0.855


**Problem 3** Use the car data and build a regression tree. Use out of sample metrics to determine the best depth for the tree. Using Feature importances, what variables are considered to be most significant. How does this compare to the linear regression model?

In [11]:
from sklearn.tree import DecisionTreeRegressor

# try different depths
best_depth = 0
best_rmse = float('inf')
results = []

for depth in range(1, 11):
    tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    y_pred_tree = tree.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_tree))
    r2 = r2_score(y_test, y_pred_tree)
    results.append((depth, rmse, r2))
    if rmse < best_rmse:
        best_rmse = rmse
        best_depth = depth

print("Depth\tRMSE\tR²")
for depth, rmse, r2 in results:
    print(f"{depth}\t{rmse:.3f}\t{r2:.3f}")

print(f"\nBest depth: {best_depth} with RMSE = {best_rmse:.3f}")

# fit best model and get feature importances
best_tree = DecisionTreeRegressor(max_depth=best_depth, random_state=42)
best_tree.fit(X_train, y_train)
importances = pd.Series(best_tree.feature_importances_, index=X.columns).sort_values(ascending=False)

print("\nFeature Importances:")
print(importances)

Depth	RMSE	R²
1	5.107	0.489
2	4.611	0.583
3	3.695	0.733
4	3.416	0.771
5	3.411	0.772
6	3.158	0.805
7	3.237	0.795
8	3.196	0.800
9	3.377	0.777
10	3.481	0.763

Best depth: 6 with RMSE = 3.158

Feature Importances:
displacement    0.652588
horsepower      0.175287
model_year      0.100389
weight          0.053929
acceleration    0.013532
cylinder        0.004276
dtype: float64


The regression tree considers displacement the most important variable, followed by horsepower and model_year. In the linear regression model, weight and model_year were the most significant. Both models agree that model_year is super important! The tree picked up on displacement and horsepower being key factors, while the linear model focused more on weight.

**Problem 4**: Consider a new American car with a 6 cylinders, a displacement of 200, a horsepower of 100, a weight of 3600, an acceleration of 14, and a model year of 83.

Part a. For the best regression model in Problem 2, and the best regression tree model in Problem 3, determine the prediction for the new car.

In [19]:

new_car_data = {
    'const': [1.0],
    'cylinder': [6],
    'displacement': [200],
    'horsepower': [100],
    'weight': [3600],
    'acceleration': [14],
    'model_year': [83],
    'weight_squared': [3600 ** 2],
    'displacement_squared': [200 ** 2],
    'weight_x_year': [3600 * 83]
}
new_car_select_const = pd.DataFrame(new_car_data)

new_car_df = pd.DataFrame([[6, 200, 100, 3600, 14, 83]], columns=X.columns)

pred_linear = model_select.predict(new_car_select_const)[0]
pred_tree = best_tree.predict(new_car_df)[0]

print(f"Linear regression prediction: {pred_linear:.2f} mpg")
print(f"Regression tree prediction: {pred_tree:.2f} mpg")

Linear regression prediction: 22.79 mpg
Regression tree prediction: 38.00 mpg


Part b. Find the confidence interval for the expected value of the prediction in the linear regression model.

In [21]:

ci_mean = model_select.get_prediction(new_car_select_const).conf_int()
print(f"CI for expected value: [{ci_mean[0,0]:.2f}, {ci_mean[0,1]:.2f}]")

CI for expected value: [21.30, 24.28]


Part c. Find the confidence interval for the predicted value using the linear regression model

In [22]:

ci_pred = model_select.get_prediction(new_car_select_const).conf_int(obs=True)
print(f"CI for predicted value: [{ci_pred[0,0]:.2f}, {ci_pred[0,1]:.2f}]")

CI for predicted value: [16.85, 28.72]


Part d. Explain why those confidence intervals are different.

The CI for the expected value is narrower because it's estimating the average mpg for all cars with these characteristics. The CI for the predicted value is wider because it accounts for individual variation - a specific car could have better or worse fuel efficiency than the average due to random factors. Basically, predicting one specific car has more uncertainty than predicting the average!

Part e. Use bootstrapping to find a confidence interval for the prediction using the regression tree model.

In [23]:

n_bootstrap = 1000
bootstrap_preds = []

for i in range(n_bootstrap):
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_boot = X_train.iloc[indices]
    y_boot = y_train.iloc[indices]

    tree_boot = DecisionTreeRegressor(max_depth=best_depth, random_state=42)
    tree_boot.fit(X_boot, y_boot)
    pred = tree_boot.predict(new_car_df)[0]
    bootstrap_preds.append(pred)

bootstrap_ci = np.percentile(bootstrap_preds, [2.5, 97.5])
print(f"Bootstrap CI for tree prediction: [{bootstrap_ci[0]:.2f}, {bootstrap_ci[1]:.2f}]")

Bootstrap CI for tree prediction: [17.67, 38.00]


**Problem 5** Airfoils are the part of an airplane wing that allows for stability in flight due to their shape. NASA did a study of the noise level of various shapes and sizes of airfoils.  The data set for their study can be loaded using

airfoil = pd.read_table("https://richardson.byu.edu/220/airfoil.csv")

More information on this data can be found here: https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise. The target variable in the data set is labeled as Pressure. The rest are predictors.

Part a. Build and tune a linear model where you scale and standardize the data.

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression as LR

# load airfoil data - use read_csv
airfoil = pd.read_csv("https://richardson.byu.edu/220/airfoil.csv")

# separate features and target - last column is target
y_air = airfoil.iloc[:, -1].values
X_air = airfoil.iloc[:, :-1].values

# train/test split
X_air_train, X_air_test, y_air_train, y_air_test = train_test_split(X_air, y_air, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_air_train)
X_test_scaled = scaler.transform(X_air_test)

# fit the model
model_air = LR()
model_air.fit(X_train_scaled, y_air_train)

# evaluate
y_pred_air = model_air.predict(X_test_scaled)
rmse_air = np.sqrt(mean_squared_error(y_air_test, y_pred_air))
r2_air = r2_score(y_air_test, y_pred_air)

print(f"Airfoil model RMSE: {rmse_air:.3f}")
print(f"Airfoil model R²: {r2_air:.3f}")

Airfoil model RMSE: 4.704
Airfoil model R²: 0.558


Part b. The manufacturing company did just build a new airfoil. They want you to use the model to predict the noise level at various velocities and frequencies. The data to predict can be found at

airfoil_new = pd.read_csv("https://richardson.byu.edu/220/airfoil_new.csv")

Find predictions for each of the new rows.

In [26]:
# load new airfoil data
airfoil_new = pd.read_csv("https://richardson.byu.edu/220/airfoil_new.csv")

# the original model was trained on 5 features, so use only first 5 columns
X_new = airfoil_new.iloc[:, :5].values

# scale the new data
X_new_scaled = scaler.transform(X_new)

# make predictions
predictions = model_air.predict(X_new_scaled)

print("Predictions for new airfoils:")
for i, pred in enumerate(predictions):
    print(f"Row {i+1}: {pred:.2f}")

Predictions for new airfoils:
Row 1: 123.31
Row 2: 123.23
Row 3: 123.12
Row 4: 122.99
Row 5: 122.83
Row 6: 122.61
Row 7: 122.36
Row 8: 122.04
Row 9: 121.59
Row 10: 121.09
Row 11: 120.45
Row 12: 119.62
Row 13: 118.54
Row 14: 117.27
Row 15: 115.62
Row 16: 113.46
Row 17: 110.91
Row 18: 107.73
Row 19: 126.25
Row 20: 126.17
Row 21: 126.06
Row 22: 125.93
Row 23: 125.77
Row 24: 125.55
Row 25: 125.30
Row 26: 124.98
Row 27: 124.54
Row 28: 124.03
Row 29: 123.39
Row 30: 122.56
Row 31: 121.48
Row 32: 120.21
Row 33: 118.56
Row 34: 116.40
Row 35: 113.85
Row 36: 110.67


Part c. The manufacturer is exploring using longer airfoil lengths than done before. They want to use lengths of 0.4 meters and they want to use your model to predict the noise level. Are there any issues with that?

Yes, there could be issues with extrapolation. If 0.4 meters is outside the range of airfoil lengths in the original training data, the model would be predicting beyond what it has seen. Linear models can be unreliable when extrapolating because the relationship might not hold outside the training range. The model was trained on certain lengths, so using it to predict for much longer airfoils could give inaccurate results since we don't know if the linear relationship continues at that length.